In [1]:
import os

os.chdir('..')

import numpy as np
import json

import plotly.io as pio

from apps.data import *

pio.templates.default = "plotly_dark"
pio.renderers.default = "browser"

In [2]:
UNDER_OVER = 56.064

In [3]:
with open('C:/Users/jrnas/Downloads/BBGM_League_1_2220_free_agency.json', encoding='latin') as f:
    r_json = json.load(f)

In [4]:
df = player_json_to_df(r_json, keep=['ratings', 'stats'])

In [5]:
df['vorp_norm'] = (df['vorp'] / df['min']) * (32 * 82)

In [6]:
test_df = df[(df['min'] > 10) & (~df['vorp_norm'].isna())].reset_index(drop=True)
test_df['vorp_norm_wt'] = test_df['vorp_norm'] * test_df['min']
agg_df = test_df.groupby('ovr')[['vorp_norm_wt', 'min']].sum().reset_index()
agg_df['vorp_norm'] = agg_df['vorp_norm_wt'] / agg_df['min']

In [7]:
model_df_under = agg_df[agg_df['ovr'] <= UNDER_OVER].reset_index(drop=True)
poly_under = np.polyfit(model_df_under['ovr'], model_df_under['vorp_norm'], 1)

model_df_over = agg_df[agg_df['ovr'] > UNDER_OVER].reset_index(drop=True)
poly_over = np.polyfit(model_df_over['ovr'], model_df_over['vorp_norm'], 1)

df['vorp_under'] = np.polyval(poly_under, df['ovr'])
df['vorp_over'] = np.polyval(poly_over, df['ovr'])
df['vorp_pred'] = np.where(
    df['ovr'] <= UNDER_OVER,
    df['vorp_under'],
    df['vorp_over']
)

np.save('models/poly_under.npy', poly_under)
np.save('models/poly_over.npy', poly_over)

In [8]:
df['cvorp'] = df['vorp'].clip(0, )
df['vorp_pct'] = df['vorp_pred'].clip(0, ) / df[df.season.between(2024, 2220)].groupby('season').vorp.sum().mean()
df['vorp_pct_cap'] = df['vorp_pct'] * 30

In [9]:
def predict_minutes(ovr):
    # =6.6208*LN(A2) - 26.515
    pred_ini = 6.6208 * np.log(ovr) - 26.515
    pred_inverse = 1 / (1 + np.exp(-pred_ini))
    pred_final = 3000 * pred_inverse
    return pred_final

In [27]:
df['min_pred'] = df['ovr'].apply(predict_minutes)
df['vorp_total'] = df['vorp_pred'].clip(0,) * df['min_pred'] / 82 / 32
df['vorp_total_pct'] = df['vorp_total'] / df[df.season.between(2024, 2220)].groupby('season').vorp.sum().mean()
df['vorp_total_pct_cap'] = df['vorp_total_pct'] * 30

In [24]:
df[(df.season == 2024) & (df.tid == 0)].sort_values('ovr', ascending=False)[
    ['firstName', 'lastName', 'ovr', 'vorp_pred', 'vorp_total', 'vorp_pct_cap', 'vorp_total_pct_cap']]

,firstName,lastName,ovr,vorp_pred,vorp_total,vorp_pct_cap,vorp_total_pct_cap
2485,Trae,Young,67,5.335780,4.817768,0.511059,0.333351
538,Dejounte,Murray,64,4.022194,3.379924,0.385244,0.233864
539,Dejounte,Murray,62,3.146469,2.489697,0.301368,0.172267
1911,De'Andre,Hunter,61,2.708607,2.070758,0.259429,0.143280
2820,Jalen,Johnson,61,2.708607,2.070758,0.259429,0.143280
1299,Clint,Capela,59,1.832883,1.295276,0.175553,0.089623
2685,Saddiq,Bey,59,1.832883,1.295276,0.175553,0.089623
2759,Bogdan,Bogdanovic,59,1.832883,1.295276,0.175553,0.089623
3374,Onyeka,Okongwu,57,0.957158,0.616067,0.091676,0.042627
4859,Garrison,Mathews,50,-0.285780,0.000000,0.000000,0.000000


In [28]:
import plotly.express as px
#px.scatter(df, x='ovr', y='vorp_pct_cap')
px.scatter(df, x='ovr', y='vorp_total_pct_cap')

In [19]:
df[df.season.between(2024, 2220)].groupby('season').vorp.sum().mean()

313.21912442015014